In [2]:
import hashlib
import json
import numpy as np
from time import time

class Blockchain:
    def __init__(self, difficulty=2, save_to_file=False):
        self.chain = []
        self.difficulty = difficulty
        self.save_to_file = save_to_file
        self.create_block(previous_hash="0", proof=1, averaged_model=np.zeros(10))  # Genesis block

    def create_block(self, proof, previous_hash, averaged_model):
        block = {
            "index": len(self.chain) + 1,
            "timestamp": time(),
            "averaged_model": averaged_model.tolist(),
            "proof": proof,
            "previous_hash": previous_hash,
        }
        self.chain.append(block)

        if self.save_to_file:
            with open(f"block_{block['index']}.txt", "w") as file:
                file.write(json.dumps(block, indent=4))
        return block

    def proof_of_work(self, previous_proof):
        proof = 0
        while not self.valid_proof(previous_proof, proof):
            proof += 1
        return proof

    def valid_proof(self, previous_proof, proof):
        guess = f"{previous_proof}{proof}".encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:self.difficulty] == "0" * self.difficulty

    @property
    def last_block(self):
        return self.chain[-1]


In [3]:
class FederatedClient:
    def __init__(self, client_id, blockchain, model_size=10):
        self.client_id = client_id
        self.blockchain = blockchain
        self.local_model = np.zeros(model_size)  # Initialize the local model

    def train(self):
        # Simulate local training by adding random noise
        self.local_model += np.random.rand(len(self.local_model)) * 0.1
        return self.local_model

    def compute_average(self, client_updates):
        # Average all client updates
        return np.mean(client_updates, axis=0)

    def mine_block(self, averaged_model):
        last_proof = self.blockchain.last_block["proof"]
        proof = self.blockchain.proof_of_work(last_proof)
        previous_hash = self.blockchain.last_block["previous_hash"]
        return self.blockchain.create_block(proof, previous_hash, averaged_model)


In [4]:
import flwr as fl

# Initialize the blockchain
blockchain = Blockchain(difficulty=4, save_to_file=True)

# Create clients
NUM_CLIENTS = 5
clients = [FederatedClient(client_id=i, blockchain=blockchain) for i in range(NUM_CLIENTS)]

2024-12-22 19:56:58,180	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [5]:

# Flower server strategy
class FederatedStrategy(fl.server.strategy.FedAvg):
    def aggregate_fit(self, rnd, results, failures):
        """Aggregate model updates and mine a block."""
        if failures:
            print(f"Round {rnd}: {len(failures)} client failures.")

        # Collect weights from clients
        client_updates = [res.parameters.tensors for _, res in results]

        # Compute average (FedAvg)
        averaged_model = np.mean(client_updates, axis=0)

        # Mine a block
        mined_block = clients[0].mine_block(averaged_model)  # First client mines the block
        print(f"Round {rnd}: Block {mined_block['index']} mined with proof {mined_block['proof']}.")

        return averaged_model, {}


In [6]:

# Flower client definition
class FederatedClientFL(fl.client.NumPyClient):
    def __init__(self, client):
        self.client = client

    def get_parameters(self):
        return self.client.local_model

    def fit(self, parameters, config):
        # Set model parameters
        self.client.local_model = parameters
        # Train locally
        self.client.train()
        return self.client.local_model, len(self.client.local_model), {}

    def evaluate(self, parameters, config):
        return 0.0, len(self.client.local_model), {}


In [7]:
def start_simulation():
    from flwr.server import ServerConfig

    # Define the number of rounds in a ServerConfig object
    server_config = ServerConfig(num_rounds=3)

    # Start the server with the strategy and configuration
    strategy = FederatedStrategy()
    fl.server.start_server(strategy=strategy, config=server_config)


In [8]:
# Start Flower clients
def start_clients():
    fl.client.start_numpy_client("localhost:8080", client=FederatedClientFL(clients[0]))

In [9]:
start_simulation()

	Instead, use the `flower-superlink` CLI command to start a SuperLink as shown below:

		$ flower-superlink --insecure

	To view usage and all available options, run:

		$ flower-superlink --help

	Using `start_server()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower server, config: num_rounds=3, no round_timeout
INFO :      Flower ECE: gRPC server running (3 rounds), SSL is disabled
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client


KeyboardInterrupt: 

In [ ]:
start_clients()